# 解析 batch_result JSONL 为 DataFrame

用途：
- 读取 `中间文件/batch_return_files/batch_result_batch_1964900313258266624.jsonl`
- 将 JSONL 解析为 Pandas DataFrame（兼容部分不规范行）
- 预览数据基本信息

运行下方所有代码单元即可得到 `df` 数据集。


In [ ]:
from __future__ import annotations

from pathlib import Path
import json
from json import JSONDecodeError
from typing import Any, Dict, List, Tuple

import pandas as pd

# # 更友好的显示设置
# pd.set_option("display.max_columns", 100)
# pd.set_option("display.width", 160)
# pd.set_option("display.max_colwidth", 200)

# # JSONL 文件路径（相对当前工作目录）
# project_root = Path.cwd()
# jsonl_path = project_root / "中间文件" / "batch_return_files" / "batch_result_batch_1964959511140634624.jsonl"
# print(f"JSONL path: {jsonl_path}")
# assert jsonl_path.exists(), f"文件不存在: {jsonl_path}"


def _extract_json_substring(text: str) -> str | None:
    """在一行文本中提取第一个完整的 { ... } JSON 子串。"""
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        return text[start : end + 1]
    return None


def parse_json_line(line: str) -> Tuple[Dict[str, Any] | None, str | None]:
    """尝试将单行 JSONL 文本解析为字典。

    返回: (obj, err)
      - obj: 解析成功的字典
      - err: 解析失败的错误信息字符串
    """
    raw = line.strip()
    if not raw:
        return None, None

    # 常见场景：SSE 或日志前缀，如 "data: {...}"
    if raw.startswith("data:"):
        raw = raw[5:].strip()

    # 尝试直接解析
    try:
        return json.loads(raw), None
    except JSONDecodeError as e1:
        # 尝试从文本中提取 JSON 子串
        candidate = _extract_json_substring(raw)
        if candidate:
            try:
                return json.loads(candidate), None
            except JSONDecodeError as e2:
                return None, f"JSONDecodeError after substring extract: {e2} | line sample: {raw[:200]}"
        return None, f"JSONDecodeError: {e1} | line sample: {raw[:200]}"


def load_jsonl(path: Path) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
    """逐行解析 JSONL，返回 (records, errors)。

    errors: [{line_no, error, line_sample}] 列表
    """
    records: List[Dict[str, Any]] = []
    errors: List[Dict[str, Any]] = []

    with path.open("r", encoding="utf-8-sig", errors="replace") as f:
        for idx, line in enumerate(f, start=1):
            obj, err = parse_json_line(line)
            if obj is not None:
                records.append(obj)
            elif err:
                errors.append({
                    "line_no": idx,
                    "error": err,
                })
            # 空行则忽略
    return records, errors


JSONL path: d:\BaiduNetdiskDownload\招聘数据\merge_listed\中间文件\batch_return_files\batch_result_batch_1964959511140634624.jsonl


AssertionError: 文件不存在: d:\BaiduNetdiskDownload\招聘数据\merge_listed\中间文件\batch_return_files\batch_result_batch_1964959511140634624.jsonl

In [2]:
# 读取与构建 DataFrame
# 读取中间文件\batch_return_files中的所有文件名称
from pathlib import Path

batch_return_dir = Path("中间文件/batch_return_files")
path_list = []

if batch_return_dir.exists() and batch_return_dir.is_dir():
    path_list = [f for f in batch_return_dir.iterdir() if f.is_file()]
    print(f"Found {len(path_list)} files in batch_return_files directory")
else:
    print("batch_return_files directory not found")

# 假设我们要处理第一个文件（如果存在的话）
if path_list:
    jsonl_path = path_list[0]
    print(f"Processing file: {jsonl_path}")
else:
    # 如果没有找到文件，使用默认路径
    jsonl_path = Path("中间文件/batch_return_files/default.jsonl")
    print("No files found, using default path")

Found 77 files in batch_return_files directory
Processing file: 中间文件\batch_return_files\batch_result_batch_1964897795249676288.jsonl


In [3]:
from tqdm import tqdm

all_records = []
all_errors = []

# 循环读取path_list中的所有文件
for jsonl_path in tqdm(path_list, desc="Processing files"):
    records, errors = load_jsonl(jsonl_path)
    
    # 合并到总列表中
    all_records.extend(records)
    all_errors.extend(errors)

print(f"Total parsed records: {len(all_records)} | Total errors: {len(all_errors)}")

df = pd.json_normalize(all_records, max_level=1)
print(df.shape)
df.head(3)


Processing files: 100%|██████████| 77/77 [00:53<00:00,  1.43it/s]


Total parsed records: 3813694 | Total errors: 0
(3813694, 4)


,custom_id,id,response.status_code,response.body
0,desc-0001308,batch_1964897795249676288,200,"{'created': 1757303575, 'usage': {'completion_tokens': 26, 'prompt_tokens': 127, 'total_tokens': 153}, 'model': 'glm-4-flash', 'id': '20250908115254197aa7871f954124', 'choices': [{'finish_reason':..."
1,desc-0001344,batch_1964897795249676288,200,"{'created': 1757303577, 'usage': {'completion_tokens': 109, 'prompt_tokens': 279, 'total_tokens': 388}, 'model': 'glm-4-flash', 'id': '20250908115254a5244eb6917b417e', 'choices': [{'finish_reason'..."
2,desc-0001352,batch_1964897795249676288,200,"{'created': 1757303575, 'usage': {'completion_tokens': 52, 'prompt_tokens': 192, 'total_tokens': 244}, 'model': 'glm-4-flash', 'id': '2025090811525421f560e491704cd1', 'choices': [{'finish_reason':..."


In [5]:
# 解析大模型输出为 Python 列表列 `responsibilities`
from typing import Any, Dict, List, Tuple
from json_repair import repair_json
from tqdm import tqdm
# json_repair 安装与导入（若不可用则跳过）



def parse_content_to_responsibilities(content: Any) -> Tuple[List[str] | None, str | None]:
    if content is None:
        return None, "no_content"

    # 如果已经是 dict，直接从字段中取
    if isinstance(content, dict):
        if "responsibilities" in content and isinstance(content["responsibilities"], list):
            items = [str(x).strip() for x in content["responsibilities"]]
            items = [x for x in items if x]
            return items, None
        # 有些模型会嵌套 content 字段
        if "content" in content:
            return parse_content_to_responsibilities(content["content"])
        return None, "dict_without_responsibilities"

    # 如果是字符串，通常是一个 JSON 字符串，先尝试 json.loads
    if isinstance(content, str):
        s = content.strip()
        try:
            loaded = json.loads(s)
        except Exception as e1:
            # 先尝试从字符串中提取 {...} 子串
            candidate = _extract_json_substring(s)
            # 1) 直接解析子串
            if candidate:
                try:
                    loaded = json.loads(candidate)
                except Exception:
                    loaded = None
            else:
                loaded = None

            # 2) 如果仍失败并且可用，尝试 json_repair 修复（对子串优先）
            if loaded is None:
                try:
                    to_repair = candidate if candidate else s
                    if repair_json is None:
                        raise RuntimeError("json_repair_not_available")
                    repaired = repair_json(to_repair)
                    loaded = json.loads(repaired)
                except Exception as e_repair:
                    return None, f"json_repair_failed:{e_repair}"
        return parse_content_to_responsibilities(loaded)

    return None, f"unsupported_content_type:{type(content).__name__}"


def parse_responsibilities_from_body(body: Any) -> Tuple[List[str] | None, str | None]:
    if not isinstance(body, dict):
        return None, "body_not_dict"

    # 优先从 choices[0].message.content 提取
    content = None
    choices = body.get("choices")
    if isinstance(choices, list) and len(choices) > 0 and isinstance(choices[0], dict):
        first = choices[0]
        msg = first.get("message")
        if isinstance(msg, dict):
            content = msg.get("content")
        if content is None:
            content = first.get("content")
        if content is None and isinstance(first.get("delta"), dict):
            content = first["delta"].get("content")

    # 其他可能位置
    if content is None:
        message_field = body.get("message") if isinstance(body.get("message"), dict) else {}
        content = (
            body.get("output_text")
            or body.get("content")
            or (message_field.get("content") if isinstance(message_field, dict) else None)
        )

    return parse_content_to_responsibilities(content)


# 仅解析 status_code == 200 的行
mask_ok = df["response.status_code"] == 200
print(f"Parsing responsibilities for {mask_ok.sum()} rows with status_code==200...")

# 使用 tqdm 添加进度条
tqdm.pandas(desc="Parsing responsibilities")
parsed_series = df.loc[mask_ok, "response.body"].progress_apply(parse_responsibilities_from_body)

df.loc[mask_ok, "responsibilities"] = parsed_series.map(lambda t: t[0])
df.loc[mask_ok, "responsibilities_error"] = parsed_series.map(lambda t: t[1])

# 统计与预览
num_ok = int(mask_ok.sum())
# num_parsed = int(df.loc[mask_ok, "responsibilities"].apply(lambda x: isinstance(x, list) and len(x) > 0).sum())
num_parsed = int(df.loc[mask_ok, "responsibilities"].apply(lambda x: isinstance(x, list)).sum())
print(f"Rows with status_code==200: {num_ok} | Parsed responsibilities: {num_parsed}")

cols = [c for c in ["custom_id", "id", "responsibilities"] if c in df.columns]
df[cols].head(10)


Parsing responsibilities for 3813694 rows with status_code==200...


Parsing responsibilities: 100%|██████████| 3813694/3813694 [00:10<00:00, 354760.05it/s]


Rows with status_code==200: 3813694 | Parsed responsibilities: 3812561


,custom_id,id,responsibilities
0,desc-0001308,batch_1964897795249676288,"[负责成品的组装、包装, 对组装工艺进行优化, 组装出现问题事汇总]"
1,desc-0001344,batch_1964897795249676288,"[负责公司内部局域网维护, 进行交换机、服务器、路由器等设备管理，以及网络平台的运行监控和维护, 负责公司内办公相关设备(电脑、打印机、传真机、考勤机、投影仪等）的维护和保养, 负责病毒的查杀，维护网络系统安全, 处理网络及计算机故障, 负责内部信息系统建设、维护, 负责电话交换机、内部线路的维护，保证畅通，维护电话系统, linux/windows/mac系统常规应用的服务的配置管理等]"
2,desc-0001352,batch_1964897795249676288,"[负责董事长的工作日程安排, 组织和安排各类会议，撰写和整理会议纪要, 负责文件的处理与存档等, 处理电话、邮件、传真等各种渠道的信息, 协助完成对外联络事宜]"
3,desc-0001385,batch_1964897795249676288,"[负责针对销售和代理商的产品培训, 收集国内外同行业的学术信息, 国内学术会议的协助、筹备与实施, 产品上市后临床试验的跟进、数据监察和回收, 培训学院的工作参与：招生、服务与客户接待, 陪同国外专家进行学术交流]"
4,desc-0001462,batch_1964897795249676288,"[收集整理用户需求，完成需求收集、确认需求和需求分析工作，编制《用户需求分析》, 组织需求分析，形成产品需求用例，制作页面原型及编写相关文档，并协助研发人员对需求进行理解, 负责需求跟踪与维护，进行需求变更控制, 跟踪产品发展趋势，保证产品设计优势, 引导设计师完成界面设计，追求UI美感与易用，达到极致体验, 协调技术开发人员，跟踪产品开发进度，完成产品的开发、测试、产品上线等工作]"
5,desc-0001467,batch_1964897795249676288,"[为银行持卡客户办理账单分期、卡片升级、客户关怀等业务, 严格执行公司制定的各项业务流程, 提升客户满意度]"
6,desc-0001516,batch_1964897795249676288,[]
7,desc-0001549,batch_1964897795249676288,"[负责对公司计算机硬件、软件系统进行建设及维护, 负责B/S架构下的产品开发工作, 参与系统应用和功能模块的设计与程序开发，单元测试，维护文档的编定, 参与公司重大信息化项目的调研、设计以及二次开发借口的维护和调试]"
8,desc-0001562,batch_1964897795249676288,"[负责输配电系统的建设、技术指导、运行管理工作, 负责对电气系统的故障分析和处理工作, 负责电气设备的选型、招投标、采购、安装、调试工作, 对电气系统的运行进行安全评估，制定安全策略, 负责制定电气设备的操作规程和管理制度, 负责输配电系统的设计工作, 负责电气设备故障的处理及日常电气设备的维修保养工作]"
9,desc-0001591,batch_1964897795249676288,"[在部门经理的领导下，管理呼叫中心客服团队, 负责策划、完善呼叫中心实施方案、管理制度、业务标准及流程，根据业务发展的实际情况，调整和完善项目内部的管理流程和规范, 管理呼叫中心客服整体业务运营，合理调配客服坐席资源，监控运营质量，不断提升呼叫中心整体业绩, 跟踪业务流程、系统化和部门间的工作协调，协助优化运营流程，不断提升业务准确度和服务效率，使得整个呼叫中心部门达到高水平的客户满意度,..."


In [17]:
output = df[["custom_id", "id", "responsibilities",'responsibilities_error']]
output = output[output['responsibilities_error'].isna()]
output = output[output['responsibilities'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
# 对responsibilities列中的列表进行去重处理
output['responsibilities'] = output['responsibilities'].apply(lambda x: list(dict.fromkeys(x)) if isinstance(x, list) else x)


In [6]:
# 统计错误数量
error_counts = df['responsibilities_error'].value_counts()
print(f"总错误数量: {error_counts.sum()}")
print(f"错误类型数量: {len(error_counts)}")
print("\n各类错误统计:")
error_counts

总错误数量: 1133
错误类型数量: 5

各类错误统计:


responsibilities_error
json_repair_failed:Expecting value: line 1 column 1 (char 0)    758
unsupported_content_type:list                                   343
dict_without_responsibilities                                    30
json_repair_failed:maximum recursion depth exceeded               1
json_repair_failed:string index out of range                      1
Name: count, dtype: int64

In [8]:
output['len'] = output['responsibilities'].apply(lambda x: len(x))

In [9]:
output['len'].value_counts().sort_index()

len
1       98635
2      190189
3      492816
4      714133
5      703666
        ...  
92          2
103         1
111         1
117         1
136         1
Name: count, Length: 86, dtype: int64

In [10]:
output[1500000:][['responsibilities']].to_parquet('list_par2.parquet')

In [25]:
len(output[:1500000]['responsibilities'].explode().to_list())

7704638

In [ ]:
# 可选：导出结果
out_dir = project_root / "中间文件" / "batch_return_files"
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = out_dir / "batch_result_batch_1964900313258266624.parsed.csv"
parquet_path = out_dir / "batch_result_batch_1964900313258266624.parsed.parquet"

print(f"Writing CSV -> {csv_path}")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"Writing Parquet -> {parquet_path}")
df.to_parquet(parquet_path, index=False)


In [ ]:
# 批量解析并合并 `中间文件/batch_return_files` 下的所有 JSONL
from pathlib import Path
from typing import List
from tqdm import tqdm

jsonl_dir = project_root / "中间文件" / "batch_return_files"
all_jsonl_files: List[Path] = sorted(jsonl_dir.glob("*.jsonl"))
print(f"Found {len(all_jsonl_files)} JSONL files under {jsonl_dir}")


def process_one_jsonl(path: Path) -> pd.DataFrame:
    records, errors = load_jsonl(path)
    if not records:
        df_empty = pd.DataFrame({
            "file": [path.name],
            "parse_exception": [None],
        })
        return df_empty

    df_one = pd.json_normalize(records, max_level=1)

    # 尝试解析 responsibilities
    if "response.body" in df_one.columns:
        parsed_series = df_one["response.body"].apply(parse_responsibilities_from_body)
        df_one["responsibilities"] = parsed_series.map(lambda t: t[0])
        df_one["responsibilities_error"] = parsed_series.map(lambda t: t[1])

    df_one["file"] = path.name

    keep_cols = [
        c for c in [
            "custom_id",
            "id",
            "response.status_code",
            "responsibilities",
            "responsibilities_error",
            "file",
        ] if c in df_one.columns
    ]
    return df_one[keep_cols]


batch_frames: List[pd.DataFrame] = []
for p in tqdm(all_jsonl_files, desc="Parsing JSONL files"):
    try:
        batch_frames.append(process_one_jsonl(p))
    except Exception as e:
        batch_frames.append(pd.DataFrame({
            "file": [p.name],
            "parse_exception": [str(e)],
        }))

merged_all = pd.concat(batch_frames, ignore_index=True, sort=False) if batch_frames else pd.DataFrame()
print("Merged shape:", merged_all.shape)

num_rows = len(merged_all)
num_resp = int(merged_all.get("responsibilities", pd.Series(dtype=object)).apply(lambda x: isinstance(x, list)).sum()) if not merged_all.empty else 0
print(f"Total rows: {num_rows} | Rows with parsed responsibilities (is list): {num_resp}")


In [ ]:
# 去重、筛选与导出
import math

if 'merged_all' not in globals():
    raise RuntimeError("merged_all 未定义，请先运行上一个合并单元格。")

if merged_all.empty:
    print("merged_all is empty; nothing to export.")
else:
    dfm = merged_all.copy()

    # 判定 responsibilities 是否为非空列表
    def _is_non_empty_list(x):
        return isinstance(x, list) and len(x) > 0

    dfm['is_list'] = dfm.get('responsibilities', pd.Series([None]*len(dfm))).apply(_is_non_empty_list)
    dfm['len_list'] = dfm.get('responsibilities', pd.Series([None]*len(dfm))).apply(lambda x: len(x) if isinstance(x, list) else -1)

    # 选主键列
    key_col = 'custom_id' if 'custom_id' in dfm.columns else ('id' if 'id' in dfm.columns else None)
    if key_col is None:
        key_col = '_row_index'
        dfm[key_col] = dfm.index.astype(str)

    # 标记 200 OK
    status = dfm.get('response.status_code', pd.Series([-1]*len(dfm)))
    dfm['_is_ok'] = status.eq(200)

    # 排序：优先 200 且 responsibilities 更长
    dfm_sorted = dfm.sort_values(by=['_is_ok', 'is_list', 'len_list'], ascending=[False, False, False])

    # 去重保留最佳行
    best_rows = dfm_sorted.drop_duplicates(subset=[key_col], keep='first').copy()

    # 导出两份：
    # 1) 仅保留解析出 responsibilities 的记录
    parsed = best_rows[best_rows['is_list']].copy()

    # 2) 全量最佳（含未解析者），便于排错
    best_all = best_rows.copy()

    # 选择导出列（存在则导出）
    export_cols = [c for c in [key_col, 'custom_id', 'id', 'responsibilities', 'responsibilities_error', 'response.status_code', 'file'] if c in best_rows.columns]

    out_dir = project_root / '中间文件'
    out_dir.mkdir(parents=True, exist_ok=True)

    out_parsed_csv = out_dir / 'batch_return_merged.csv'
    out_parsed_parquet = out_dir / 'batch_return_merged.parquet'
    out_best_all_parquet = out_dir / 'batch_return_merged_all.parquet'

    print(f"Saving parsed-only CSV -> {out_parsed_csv}")
    parsed[export_cols].to_csv(out_parsed_csv, index=False, encoding='utf-8-sig')

    print(f"Saving parsed-only Parquet -> {out_parsed_parquet}")
    parsed[export_cols].to_parquet(out_parsed_parquet, index=False)

    print(f"Saving best-all Parquet -> {out_best_all_parquet}")
    best_all[export_cols].to_parquet(out_best_all_parquet, index=False)

    print(
        f"Done. parsed-only rows: {len(parsed)} / best-all rows: {len(best_all)} | key_col: {key_col}"
    )


In [7]:
import pandas as pd
df1 = pd.read_parquet('list_par_with_matches.parquet')
df2 = pd.read_parquet('list_par2_with_matches.parquet')
dfx = pd.concat([df1,df2])

In [35]:
output = output.reset_index(drop=True)
dfx = dfx.reset_index(drop=True)
output.drop('responsibilities',inplace=True,axis =1)
merged = pd.concat([output,dfx],axis = 1)
merged = merged[['custom_id', 'responsibilities', 'responsibilities_match']]

In [37]:
merged.to_parquet('id_responsibility_match.parquet')